In [1]:
!python3 labelstudio2doccano.py --labelstudio_file test_label_studio.json

In [2]:
!pip3 list

Package                       Version
----------------------------- --------------
aniso8601                     9.0.1
ansi2html                     1.8.0
anyio                         3.6.2
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astor                         0.8.1
asttokens                     2.2.1
attrs                         22.2.0
awscli                        1.22.101
Babel                         2.11.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
bokeh                         2.4.3
boto3                         1.26.79
botocore                      1.29.79
brotlipy                      0.7.0
captum                        0.5.0
catalogue                     2.0.8
certifi                       2022.12.7
cffi                          1.15.1
charset-normalizer            2.1.1


In [4]:
!python3 doccano.py \
    --doccano_file doccano_ext.jsonl \
    --task_type ext \
    --save_dir .\
    --splits 0.7 0.2 0.1 \
    --schema_lang ch 

/opt/conda/envs/pytorch/lib/python3.9/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/opt/conda/envs/pytorch/lib/python3.9/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/opt/conda/envs/pytorch/lib/python3.9/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('ruamel')`.
Implementing impli